# 🔢 05. 정수 인코딩 (Integer Encoding): 컴퓨터에게 숫자 이름표 달아주기

## 1. 개요
컴퓨터는 '사과', '바나나' 같은 글자를 이해하지 못합니다. 오직 **숫자**만 계산할 수 있죠.
그래서 우리는 단어 하나하나에 **고유한 번호(출석 번호)**를 붙여줄 것입니다.

> **쉬운 비유**:  
> - 교실에 학생이 30명 있습니다.
> - 선생님이 부를 때 "김철수!"라고 부를 수도 있지만, 행정 처리를 위해 **"1번!"** 이라고 부르는 것이 더 효율적일 때가 많습니다.
> - 이처럼 단어(학생)에게 번호(출석 번호)를 부여하는 과정입니다.

---

## 2. 작업 순서 (레시피)

1. **문장 쪼개기 (토큰화)**: 글을 단어 조각으로 나눕니다.
2. **단어 정리하기 (빈도수 계산)**: 어떤 단어가 몇 번 나왔는지 셉니다.
3. **번호 매기기 (인덱싱)**: 
    - 많이 나온 단어일수록 **앞 번호(1번, 2번...)**를 줍니다.
    - 왜요? 자주 쓰는 단어를 작은 숫자로 두면 메모리 효율에 좋기 때문입니다.
4. **변환하기 (인코딩)**: 문장의 단어들을 숫자로 싹 바꿉니다.

### 실습 데이터 준비
어린 왕자(The Little Prince) 텍스트를 일부 가져와서 연습해봅시다.

In [ ]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

### 전처리 (청소하기)
- 문장을 단어로 쪼개고 (`tokenize`)
- 대소문자를 통일하고 (`lower`)
- 쓸모없는 단어를 버립니다 (`stopword removal`)

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# 0. 필요한 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')

# 1. 문장 단위로 먼저 나누기
sentences = sent_tokenize(raw_text)

# 2. 단어 단위로 나누고 청소하기
vocab = {}  # 단어장 (단어: 등장횟수)
preprocessed_sentences = [] # 청소된 문장들을 모을 리스트
stop_words = set(stopwords.words('english'))

for sentence in sentences:
    # 소문자 변환 + 단어 토큰화
    tokens = word_tokenize(sentence.lower())
    
    cleaned_tokens = []
    for token in tokens:
        # 불용어(the, is 등) 빼고, 길이 2보다 큰 단어만 남기기
        if token not in stop_words and len(token) > 2:
            cleaned_tokens.append(token)
            
            # 단어장(vocab)에 횟수 기록
            if token not in vocab:
                vocab[token] = 0
            vocab[token] += 1
            
    preprocessed_sentences.append(cleaned_tokens)

print("청소된 첫 문장:", preprocessed_sentences[0])
print("발견된 총 단어 개수:", len(vocab))

### 3. 번호표 배부 (빈도수 정렬 & 인덱싱)
가장 많이 등장한 단어부터 1번을 줍니다.

In [ ]:
# 1. 많이 나온 순서대로 정렬 (내림차순)
# vocab.items()는 (단어, 횟수) 쌍을 줍니다. -> x[1]은 횟수!
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

print("가장 많이 나온 단어 Top 5:", vocab_sorted[:5])

# 2. 번호 매기기
word_to_index = {}
i = 0

for (word, frequency) in vocab_sorted:
    # 빈도수가 너무 적은(1번만 나온) 단어는 제외해볼까요?
    if frequency > 1:
        i = i + 1
        word_to_index[word] = i

print("\n번호표 배부 완료! (일부만 출력)")
# 앞에서 5개만 출력해보기
# 딕셔너리는 슬라이싱이 안 돼서 리스트로 바꿔서 출력
print(list(word_to_index.items())[:5])

### ⚠️ OOV (Out-Of-Vocabulary): 모르는 단어 처리

만약 단어장(word_to_index)에 없는 새로운 단어가 나오면 어떻게 할까요?
그럴 땐 **"기타 등등"**을 의미하는 특별한 번호를 줍니다.
보통 `OOV` 또는 `UNK` (Unknown)라는 이름을 쓰고, 가장 마지막 번호를 줍니다.

In [ ]:
vocab_size = len(word_to_index)

# 단어장 맨 끝 번호 + 1을 OOV 번호로 지정
word_to_index['OOV'] = vocab_size + 1

print("OOV의 번호:", word_to_index['OOV'])

## 4. 텍스트를 숫자로 변환 (Encoding)

이제 단어장을 보면서 문장의 단어들을 번호로 바꿔치기합니다.

In [ ]:
encoded_sentences = []

for sentence in preprocessed_sentences:
    encoded_sentence = []
    for word in sentence:
        try:
            # 단어장에 있으면 그 번호를 가져옴
            encoded_sentence.append(word_to_index[word])
        except KeyError:
            # 단어장에 없으면 OOV 번호를 가져옴
            encoded_sentence.append(word_to_index['OOV'])
            
    encoded_sentences.append(encoded_sentence)

print("원본:", preprocessed_sentences[0])
print("변환:", encoded_sentences[0])

## 5. 꿀팁: Keras Tokenizer (자동화 도구)

위 과정을 파이썬 for문으로 짜면 복잡하죠? 
**Keras**라는 라이브러리를 쓰면 이 모든 걸 2줄로 끝낼 수 있습니다. (실무에선 이걸 씁니다!)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 단순 예시 문장들
sentences = [
    'I love my dog',
    'I love my cat',
    'You love my dog!',
    'Do you think my dog is amazing?'
]

# 1. 토크나이저 생성 (OOV 토큰 이름 지정)
tokenizer = Tokenizer(oov_token="<OOV>")

# 2. 단어장 만들기 (fit_on_texts)
# 문장들을 주면 알아서 빈도수 계산하고 번호 매깁니다.
tokenizer.fit_on_texts(sentences)

# 단어장 확인 (word_index)
print("단어장:", tokenizer.word_index)

# 3. 숫자로 변환하기 (texts_to_sequences)
sequences = tokenizer.texts_to_sequences(sentences)

print("\n변환 결과:")
for seq in sequences:
    print(seq)

## 마무리 요약

1. 컴퓨터를 위해 글자에 **번호표(Index)**를 붙입니다.
2. **자주 나오는 단어**일수록 앞 번호(1, 2...)를 줍니다.
3. 모르는 단어는 **OOV**라는 퉁치는 번호로 처리합니다.
4. 실전에서는 **Keras Tokenizer**를 쓰면 매우 편합니다.